# Task Exercise 4.6

In [57]:
# importing libraries
import pandas as pd
import numpy as np
import os

In [58]:
# importing previously merged data frame
path = r'C:\Users\muham\OneDrive\Desktop\careerfoundry\Data Immersion\Achievement 4\15-08-2025 Instacart Basket Analysis'
df_merged_large = pd.read_pickle(os.path.join(path, '02 Data', 'prepared Data', 'orders_products_combined.pkl'))

In [59]:
df_merged_large.shape

(32434489, 13)

In [60]:
# importing clean products data frame
df_prods_clean = pd.read_csv(os.path.join(path, '02 Data', 'prepared Data', 'products_checked.csv'), index_col = False)

In [61]:
df_prods_clean.shape

(49672, 5)

In [62]:
df_prods_clean.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [63]:
df_prods_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49672 entries, 0 to 49671
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49672 non-null  int64  
 1   product_name   49672 non-null  object 
 2   aisle_id       49672 non-null  int64  
 3   department_id  49672 non-null  int64  
 4   prices         49672 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.9+ MB


In [64]:
# duplicate product keys
dupes = df_prods_clean[df_prods_clean.duplicated('product_id', keep=False)].sort_values('product_id')

In [65]:
dupes

,product_id,product_name,aisle_id,department_id,prices
6784,6800,Revive Zero Vitamin Water,64,7,6.4
6785,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26504,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26505,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [66]:
# keep one row per product_id
df_prods_key = (df_prods_clean
.sort_values(['product_id'])
.drop_duplicates(subset='product_id', keep='first'))

In [67]:
df_prods_key.shape

(49670, 5)

In [68]:
df_merged_large.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_first_order,product_id,add_to_cart_order,reordered,_merge
0,0,0,2539329,1,1,2,8,NaN,1,196,1,0,both
1,0,0,2539329,1,1,2,8,NaN,1,14084,2,0,both
2,0,0,2539329,1,1,2,8,NaN,1,12427,3,0,both
3,0,0,2539329,1,1,2,8,NaN,1,26088,4,0,both
4,0,0,2539329,1,1,2,8,NaN,1,26405,5,0,both


In [69]:
# dropping unnamed columns
df_merged_large = df_merged_large.loc[:, ~df_merged_large.columns.str.contains("^Unnamed")]
df_merged_large = df_merged_large.loc[:, ~df_merged_large.columns.str.contains("^_merge")]

In [70]:
df_merged_large.shape

(32434489, 10)

In [71]:
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_first_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,1,196,1,0
1,2539329,1,1,2,8,NaN,1,14084,2,0
2,2539329,1,1,2,8,NaN,1,12427,3,0
3,2539329,1,1,2,8,NaN,1,26088,4,0
4,2539329,1,1,2,8,NaN,1,26405,5,0


In [72]:
# merging df_merged_large and df_prods_key(clean products data frame)
ords_prods_merge = df_merged_large.merge(
df_prods_key,
on='product_id',
how='left',
indicator=True,
validate='m:1' # many orders to one product
)

In [73]:
ords_prods_merge['_merge'].value_counts()

_merge
both          32404289
left_only        30200
right_only           0
Name: count, dtype: int64

In [74]:
ords_prods_merge.shape

(32434489, 15)

In [75]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 15 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   is_first_order          int64   
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                float64 
 12  department_id           float64 
 13  prices                  float64 
 14  _merge                  category
dtypes: category(1), float64(4), int64(9), object(1)
memory usage: 3.4+ GB


In [76]:
ords_prods_merge_innerjoin = ords_prods_merge.query("_merge == 'both'").drop(columns=['_merge'])

In [77]:
ords_prods_merge_innerjoin.shape

(32404289, 14)

In [78]:
ords_prods_merge_innerjoin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32404289 entries, 0 to 32434488
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   is_first_order          int64  
 7   product_id              int64  
 8   add_to_cart_order       int64  
 9   reordered               int64  
 10  product_name            object 
 11  aisle_id                float64
 12  department_id           float64
 13  prices                  float64
dtypes: float64(4), int64(9), object(1)
memory usage: 3.6+ GB


In [79]:
# cleaning up data types 
for c in ['aisle_id', 'department_id']:
  ords_prods_merge_innerjoin[c] = ords_prods_merge_innerjoin[c].astype('Int64')

In [ ]:
 # checking for merge rate 
ords_prods_merge_outerjoin = df_merged_large.merge(df_prods_key, on = 'product_id', how = 'outer', indicator = True)

In [54]:
ords_prods_merge_outerjoin['_merge'].value_counts()

_merge
both          32404289
left_only        30200
right_only          11
Name: count, dtype: int64

In [84]:
# exporting the merged data frame
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_leftjoin.pkl'))

In [85]:
ords_prods_merge_innerjoin.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_innerjoin.pkl'))